<a href="https://colab.research.google.com/github/CT2020Hypernym/hyponym_and_hyperonym/blob/master/Tatiana/Pos_Neg_ruWordNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# import pandas, BeautifulSoup, tqdm
import pandas as pd
from bs4 import BeautifulSoup as b
from tqdm import tqdm

# Convert XML to CSV with synset relations for Nouns

In [3]:
# access to google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# read XML file and find all "relations"
with open('/content/drive/My Drive/Colab Notebooks/synset_relations.N.xml', "r", encoding='utf-8') as f: # opening xml file
    content = f.read()
soup = b(content, "lxml")
relations = soup.find_all('relation')

In [5]:
# print the number of noun pairs from ruWordNet 
print(len(relations))

131034


In [0]:
# parsing to CSV file with synset relations
for_csv = pd.DataFrame()
for i in range(len(relations)):
    if i%1000 == 0:
        print(i)
    for_csv = for_csv.append(relations[i].attrs, ignore_index=True)
for_csv.to_csv('/content/drive/My Drive/Colab Notebooks/synset_relations.N.csv', encoding='utf-8', header=True, index=False)

# Get ids of words from ruWordNet with relation == 'hyponym' 

In [0]:
# read .csv file
synrel = pd.read_csv('/content/drive/My Drive/Colab Notebooks/synset_relations.N.csv')

In [0]:
# print all kinds of relations
synrel.name.unique()

array(['POS-synonymy', 'hypernym', 'hyponym', 'domain', 'part holonym',
       'part meronym', 'instance hyponym', 'instance hypernym', 'antonym'],
      dtype=object)

In [7]:
# get child_id, hyponym, parent_id (for example, "кошка, hyponym, домашнее животное"; "котенок, hyponym, кошка")
hypernymy = synrel.loc[(synrel['name'] == 'hyponym')]
hypernymy

,child_id,name,parent_id
2,126551-N,hyponym,147272-N
6,5792-N,hyponym,7331-N
24,139523-N,hyponym,135214-N
33,120440-N,hyponym,112039-N
40,134577-N,hyponym,123458-N
...,...,...,...
131026,135768-N,hyponym,3949-N
131027,142357-N,hyponym,3949-N
131028,144031-N,hyponym,3949-N
131029,147153-N,hyponym,3949-N


In [0]:
# save to .csv file on google drive
hypernymy.to_csv('/content/drive/My Drive/Colab Notebooks/hypernymy.N_TAB.csv', encoding='utf-8-sig', header=True, index=False)

In [0]:
# instance hypernyms are kind of toponyms; we desided don't get them

# instance_hypernymy = synrel.loc[(synrel['name'] == 'instance hyponym')]
# instance_hypernymy

In [0]:
# save to another .csv file on google drive
# instance_hypernymy.to_csv('/content/drive/My Drive/Colab Notebooks/instance_hypernymy.N.csv', encoding='utf-8-sig', header=True, index=False)

# Convert XML to CSV with synsets for Nouns

In [12]:
# read and parse XML to CSV with synsets
with open('/content/drive/My Drive/Colab Notebooks/synsets.N.xml', "r", encoding='utf-8-sig') as f: # opening xml file
    content = f.read()
soup = b(content, "lxml")
synsets = soup.find_all('synset')
for_csv = pd.DataFrame()
for i in range(len(synsets)):
    if i%1000 == 0:
        print(i)
    for_csv = for_csv.append(synsets[i].attrs, ignore_index=True)
for_csv.to_csv('/content/drive/My Drive/Colab Notebooks/synsets.N.csv', encoding='utf-8-sig', header=True, index=False)

0
1000
2000


KeyboardInterrupt: ignored

# Collect child_names and parent_names into one file with hypernyms

In [0]:
# Из файла с гиперонимами hypernymy.N.csv по child_id находим в файле synsets.N.csv и вытаскиваем ruthes_name

In [0]:
# read CSV file with synsets
nouns = pd.read_csv('/content/drive/My Drive/Colab Notebooks/synsets.N.csv')

In [0]:
# test
# nouns.loc[nouns.id == hypernymy.loc[0,'child_id']].ruthes_name.array[0]

In [0]:
hypernymy = pd.read_csv('/content/drive/My Drive/Colab Notebooks/hypernymy.N.csv')
hypernymy_ids = hypernymy[['child_id','parent_id']]

In [0]:
child_ids = hypernymy.child_id
parent_ids = hypernymy.parent_id

In [19]:
# collect all hyponyms
for i in tqdm(range(len(child_ids)), desc='Progress'):
    hypernymy.loc[i,'child_id'] = nouns.loc[nouns.id == hypernymy.loc[i,'child_id']].ruthes_name.array[0]

Progress: 100%|██████████| 37249/37249 [04:19<00:00, 143.68it/s]


In [28]:
# collect all hypernyms
for i in tqdm(range(len(parent_ids)), desc='Progress'):
    hypernymy.loc[i,'parent_id'] = nouns.loc[nouns.id == hypernymy.loc[i,'parent_id']].ruthes_name.array[0]

Progress: 100%|██████████| 37249/37249 [04:42<00:00, 132.09it/s]


In [0]:
# save all pairs into one file
hypernymy[['child_id','parent_id']].to_csv('/content/drive/My Drive/Colab Notebooks/pos_hypernymy.N_SEP.csv', encoding='utf-8-sig', header=True, index=False, sep=';')

# Get ids of words from ruWordNet with relations other than 'hypernym'

In [38]:
# read .csv file
synrel = pd.read_csv('/content/drive/My Drive/Colab Notebooks/synset_relations.N.csv')
# print all kinds of relations
synrel.name.unique()

array(['POS-synonymy', 'hypernym', 'hyponym', 'domain', 'part holonym',
       'part meronym', 'instance hyponym', 'instance hypernym', 'antonym'],
      dtype=object)

In [44]:
# test
synrel.head()

,child_id,name,parent_id
0,147272-A,POS-synonymy,147272-N
1,4544-N,hypernym,147272-N
2,126551-N,hyponym,147272-N
3,2201-N,domain,147272-N
4,1150-N,domain,147272-N


In [0]:
# get relations other than hypernymy
without_hyps = synrel.loc[(synrel['name'] == 'part meronym') | (synrel['name'] == 'antonym') | (synrel['name'] == 'part holonym') | (synrel['name'] == 'domain')]


In [0]:
# save to file
without_hyps.to_csv('/content/drive/My Drive/Colab Notebooks/without_hyps.csv' , encoding='utf-8-sig', header=True, index=False)

# Collect relations other than 'hypernym'

In [0]:
without_hyps = pd.read_csv('/content/drive/My Drive/Colab Notebooks/without_hyps.N.csv')
without_hyps_ids = without_hyps[['child_id','parent_id']]

In [0]:
child_ids = without_hyps.child_id
parent_ids = without_hyps.parent_id

In [61]:
# test
without_hyps.loc[i,'child_id']

'2201-N'

In [62]:
for i in tqdm(range(len(child_ids)), desc='Progress'):
    without_hyps.loc[i,'child_id'] = nouns.loc[nouns.id == without_hyps.loc[i,'child_id']].ruthes_name.array[0]



Progress:   0%|          | 0/34677 [00:00<?, ?it/s]

Progress:   0%|          | 18/34677 [00:00<03:15, 176.95it/s]

Progress:   0%|          | 38/34677 [00:00<03:11, 181.34it/s]

Progress:   0%|          | 57/34677 [00:00<03:10, 182.00it/s]

Progress:   0%|          | 76/34677 [00:00<03:09, 182.28it/s]

Progress:   0%|          | 95/34677 [00:00<03:08, 183.64it/s]

Progress:   0%|          | 115/34677 [00:00<03:05, 186.67it/s]

Progress:   0%|          | 133/34677 [00:00<03:07, 184.40it/s]

Progress:   0%|          | 152/34677 [00:00<03:07, 184.53it/s]

Progress:   0%|          | 170/34677 [00:00<03:14, 177.18it/s]

Progress:   1%|          | 190/34677 [00:01<03:10, 181.20it/s]

Progress:   1%|          | 209/34677 [00:01<03:08, 182.72it/s]

Progress:   1%|          | 228/34677 [00:01<03:09, 182.24it/s]

Progress:   1%|          | 248/34677 [00:01<03:06, 185.00it/s]

Progress:   1%|          | 267/34677 [00:01<03:05, 185.37it/s]

Progress:   1%|          | 287/34677 [00:01<03:03, 187

In [63]:
for i in tqdm(range(len(without_hyps.parent_id)), desc='Progress'):
    without_hyps.loc[i,'parent_id'] = nouns.loc[nouns.id == without_hyps.loc[i,'parent_id']].ruthes_name.array[0]



Progress:   0%|          | 0/34677 [00:00<?, ?it/s]

Progress:   0%|          | 16/34677 [00:00<03:42, 155.60it/s]

Progress:   0%|          | 31/34677 [00:00<03:47, 152.02it/s]

Progress:   0%|          | 48/34677 [00:00<03:41, 156.61it/s]

Progress:   0%|          | 66/34677 [00:00<03:35, 160.48it/s]

Progress:   0%|          | 83/34677 [00:00<03:35, 160.82it/s]

Progress:   0%|          | 101/34677 [00:00<03:30, 164.53it/s]

Progress:   0%|          | 119/34677 [00:00<03:25, 168.42it/s]

Progress:   0%|          | 137/34677 [00:00<03:21, 171.24it/s]

Progress:   0%|          | 155/34677 [00:00<03:20, 172.01it/s]

Progress:   0%|          | 172/34677 [00:01<03:23, 169.17it/s]

Progress:   1%|          | 189/34677 [00:01<03:25, 167.93it/s]

Progress:   1%|          | 206/34677 [00:01<03:29, 164.20it/s]

Progress:   1%|          | 224/34677 [00:01<03:25, 167.40it/s]

Progress:   1%|          | 241/34677 [00:01<03:25, 167.95it/s]

Progress:   1%|          | 259/34677 [00:01<03:23, 168

In [33]:
without_hyps[['child_id', 'parent_id']].to_csv('/content/drive/My Drive/Colab Notebooks/neg_hypernymy.N.csv', encoding='utf-8-sig', header=True, index=False, sep=';')

NameError: ignored